In [13]:
import mikeio as mi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gdp
from shapely.geometry import Point, LineString
import sys
from math import atan2, degrees, hypot,radians,cos,sin
import shapely.wkt
import shapely.ops


def reverse_geom(geom):
    def _reverse(x, y, z=None):
        if z:
            return x[::-1], y[::-1], z[::-1]
        return x[::-1], y[::-1]

    return shapely.ops.transform(_reverse, geom)


def get_grid_size(ds):
    # Check if quadratic grid
    if ds.geometry.dx != ds.geometry.dy:
        print('The grid is not quadratic. This might cause insufficient results')
    return ds.geometry.dx


def PerpendicularFor2Points(pointA, pointB):
    changeInX = pointB[0] - pointA[0]
    changeInY = pointB[1] - pointA[1]
    return atan2(changeInY, changeInX)

def calculate_flow_magnitude(df, grid_size, perpendicular_line):
    # Get the columns of the df. The two columns that will be used for further calculations are the P flux (flow in x direction, index = 2) and Q Flux (flow in y direction, index = 3).
    columns = df.columns

    # Get the coordinates of the subline and calculate its angle. Get the perpendicular line. 
    coord_list = list(sub_line.coords)
    start_coord = coord_list[0]
    end_coord = coord_list[1]
    angle_line = PerpendicularFor2Points(start_coord, end_coord)
    perpendicular_line = angle_line + radians(90)



    # Resultant flow is given by hypothenuse of both. Note that the flow is given in per meter, meaning that we need to multiply by the grid size
    df['Discharge [meter^3/s]'] = df.apply(
        lambda x: x[columns[3]]*sin(perpendicular_line) +  x[columns[2]]*cos(perpendicular_line), axis=1)
    # df['Discharge'] = df['Discharge']
    df['flow_direction'] = df.apply(lambda x: degrees(
        atan2(x[columns[3]], x[columns[2]])), axis=1)



    # Update the columns, and calculate flow direction
    columns = df.columns
    df['Discharge [meter^3/s]'] = df.apply(lambda x: x[columns[4]] if calculate_flow_direction(
        perpendicular_line, x[columns[5]]) else x[columns[4]]*-1, axis=1)
    df['Discharge + [meter^3/s]'] = df.apply(
        lambda x: x[columns[4]] if x[columns[4]] > 0 else 0, axis=1)
    df['Discharge - [meter^3/s]'] = df.apply(
        lambda x: x[columns[4]] if x[columns[4]] < 0 else 0, axis=1)

    # Calculate the cumulative sum of the flows
    df['Cum. disc. + [meter^3]'] = np.cumsum(df['Discharge + [meter^3/s]'])
    df['Cum. disc. - [meter^3]'] = np.cumsum(df['Discharge - [meter^3/s]'])
    df['Cum. disc. [meter^3/s]'] = np.cumsum(df['Discharge [meter^3/s]'])
    return df[['time','Discharge + [meter^3/s]', 'Discharge - [meter^3/s]', 'Discharge [meter^3/s]', 'Cum. disc. + [meter^3]', 'Cum. disc. - [meter^3]', 'Cum. disc. [meter^3/s]']]


def get_values_for_linestring(grid_size, ds, geometry_row):

    # Number of points in the polyline
    num_coords = len(geometry_row.coords)

    # Convert gridsize to integer
    grid_size = int(grid_size)

    # We will take timesteps equal to grid_size/2 in order not to miss any of the cells.
    # However, in order to not duplicate any of the cells, we need to check whether the coordinates have been calculated before
    calculated_coords = []

    # Loop through each coordinate, create a new polyline for each and extract result
    for j in range(1, num_coords):

        # Define start and end coordinates
        starting_coord = geometry_row.coords[j-1]
        end_coord = geometry_row.coords[j]
        # Create new LineString of new coordinates and calculate its perpendicular angle
        sub_line = LineString([starting_coord, end_coord])
        perpendicular_angle = PerpendicularFor2Points(starting_coord,end_coord)

        # Round the length of the line to closest integer and convert
        length_of_line = int(
            np.round(Point(starting_coord).distance(Point(end_coord)), 0))

        # Loop thorugh the values in the dfs2 file, with the step size equivalent to the gridsize
        for i in range(1, length_of_line, grid_size):
            
            if i == 1:
                point_array = ds.sel(x=ip.coords[0][0], y=ip.coords[0][1])
                calculated_coords.append([point_array.geometry.x,point_array.geometry.y])
                prev_df = point_array.to_dataframe()
                
            # Get coordinates of point with i distance from start coordinate
            ip = sub_line.interpolate(distance=i)
            point_array = ds.sel(x=ip.coords[0][0], y=ip.coords[0][1])

            #REMEMBER! NOW THE IDEA IS TO TAKE THE PREV_DF AND CURR_DF AND CALCULATE THE AVERAGE!!!!!
            # Check whether the point have been calculated before
            new_coord_list = [point_array.geometry.x, point_array.geometry.y]
            if new_coord_list not in calculated_coords:
                # Create a df with this cross sections values
                df_next_coord = point_array.to_dataframe()

                # Calculate flow magnitude and its direction
                df_next_coord = calculate_flow_magnitude(
                    df_next_coord, grid_size, sub_line)
                # Concatenate the results of the new coordinate to result_df. Note that here they are concatenated vertically
                df_result = pd.concat([df_result, df_next_coord])

                # append the coordinates to calculated_coords
                calculated_coords.append(new_coord_list)





    # TEST! Include last coord.
    # Extract results
    point_array = ds.sel(x=end_coord[0], y=end_coord[1])
    new_coord_list = [point_array.geometry.x, point_array.geometry.y]

    if new_coord_list not in calculated_coords:
        # Extract results for each item in the dfs2
        # print("Du missade sista!!!")
        df_next = point_array.to_dataframe()
        df_next_coord = calculate_flow_magnitude(
            df_next, grid_size, sub_line)
        # Add the coordinates of point_array to our memory list
        df_result = pd.concat([df_result, df_next_coord])
        calculated_coords.append(new_coord_list)

    df_result = df_result.groupby('time', as_index=False).sum()
    # For swedish users, the , is the default separator, instead of . (This is a feature requested by the user)
    # columns = df_result.columns
    # for col in columns:
    #    df_result[col] = df_result[col].astype(
    #        str).str.replace('.', ',', regex=False)

    return df_result,calculated_coords,point_array


def dfs2tocsv(filepath_ds, filepath_shape,output_dir_and_name):

    # Check whether it is a dfs2 file or not
    if filepath_ds[-4:] != 'dfs2':
        print("Please pick a dfs2 file")
        return False

    # Read dfs2
    ds = mi.read(filepath_ds)

    # Check whether it is a shp file or not
    if filepath_shape[-3:] != 'shp':
        print("Please pick a shapefile")
        return False

    # Read shapefile
    shape_df = gdp.read_file(filepath_shape)

    # Check whether the file is a Polyline
    if not isinstance(shape_df['geometry'][0], LineString):
        print('The file is not a Polyline, please specify a Polyline instead.')
        return False
    
    index_check = 0
    #Check whether the ids are unique in the shapefile
    if len(shape_df['Id']) != shape_df['Id'].nunique():
        index_check = 1
        print('The values in the Id column in the shapefile is not unique. The sheets will be named after polyline index')

    # Get grid size
    grid_size = get_grid_size(ds)

    # Dictionary with results
    df_dict = {}

    # Loop through df with the shapefile and save the results
    for index, row in shape_df.iterrows():

        # Check direction. The calculations should always be done from left to right.
        first_coord = row['geometry'].coords[0]
        last_coord = row['geometry'].coords[len(row['geometry'].coords)-1]
        angle = AngleBtw2Points(first_coord, last_coord)

        # If angle is in the second or third quadrant, the general direction is right, so it must be reversed
        if (angle > 90) | (angle < -90):
            row['geometry'] = reverse_geom(row['geometry'])
            #print(
            #    'The cross section is defined from right to left. The cross section will be reversed.')
        if index_check == 1:
            df_dict[index] = get_values_for_linestring(
                grid_size, ds, row['geometry'])
        else:
            df_dict[row['Id']] = get_values_for_linestring(
                grid_size, ds, row['geometry'])
        

    # Create a multilevel indexed dataframe for easier read in the csv format (requested by users)
    #resultant_df = pd.concat(df_dict.values(), axis=1, keys=df_dict.keys())
    #resultant_df.to_csv(output_dir_and_name, sep=';')

    #Try with excelwriter
    with pd.ExcelWriter(output_dir_and_name) as writer:
        for key, value in df_dict.items():
            value.to_excel(writer,sheet_name=str(key))
    #return resultant_df


# if __name__ == '__main__':
#    dfs2tocsv(sys.argv[1], sys.argv[2], sys.argv[3])


In [11]:
filepath_ds = r'C:\Project folder\python_specialuppdrag\MASV specialuppdrag\06_Linnestaden_HPQ.dfs2'
filepath_shp = r'C:\Project folder\python_specialuppdrag\MASV specialuppdrag\test_cells.shp'
output_dir = r'C:\Project folder\python_specialuppdrag\output_csvs\heja.xls'
#test = dfs2tocsv(filepath_ds,filepath_shp)
ds = mi.read(filepath_ds)
shape_df = gdp.read_file(filepath_shp)
grid_size= get_grid_size(ds)
test,coords,point_array= get_values_for_linestring(grid_size, ds, shape_df['geometry'][0])
#dfs2tocsv(filepath_ds,filepath_shp,output_dir)

In [12]:
ds_path = r'C:\Project folder\python_specialuppdrag\MASV specialuppdrag\flux_dynamisk.dfs2'
ds = mi.read(ds_path)
test = pd.DataFrame()
for coord in coords:
    point_array = ds.sel(x=coord[0], y=coord[1]).to_dataframe()
    test = pd.concat([test,point_array],axis=1)

In [14]:
test.to_csv('from_maxflux.csv',sep=';')


In [47]:
to_shapefile = pd.DataFrame(data = {'coordinate_points':coords_list})
from shapely.geometry import Point
to_shapefile['coordinate_points'] = to_shapefile['coordinate_points'].apply(Point)
df = gdp.GeoDataFrame(to_shapefile,geometry='coordinate_points')
df.to_file(r'C:\Project folder\python_specialuppdrag\MASV specialuppdrag\endast_tvo_comp.shp',driver = 'ESRI Shapefile')

C:\Users\masv\AppData\Local\Temp\ipykernel_16000\1494209363.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df.to_file(r'C:\Project folder\python_specialuppdrag\MASV specialuppdrag\endast_tvo_comp.shp',driver = 'ESRI Shapefile')


In [12]:
test

,time,Discharge + [meter^3/s],Discharge - [meter^3/s],Discharge [meter^3/s],Cum. disc. + [meter^3],Cum. disc. - [meter^3],Cum. disc. [meter^3/s]
0,2014-07-26 17:45:00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
1,2014-07-26 17:50:00,1.414771e-09,0.000000,1.414771e-09,1.414771e-09,0.000000,1.414771e-09
2,2014-07-26 17:55:00,0.000000e+00,-0.037214,-3.721410e-02,1.414771e-09,-0.037214,-3.721410e-02
3,2014-07-26 18:00:00,0.000000e+00,-0.121917,-1.219171e-01,1.414771e-09,-0.159131,-1.591312e-01
4,2014-07-26 18:05:00,0.000000e+00,-0.347283,-3.472832e-01,1.414771e-09,-0.506414,-5.064144e-01
...,...,...,...,...,...,...,...
143,2014-07-27 05:40:00,2.126962e-21,0.000000,2.126962e-21,1.479084e-03,-14.748500,-1.474702e+01
144,2014-07-27 05:45:00,2.126962e-21,0.000000,2.126962e-21,1.479084e-03,-14.748500,-1.474702e+01
145,2014-07-27 05:50:00,2.126962e-21,0.000000,2.126962e-21,1.479084e-03,-14.748500,-1.474702e+01
146,2014-07-27 05:55:00,8.071340e-21,0.000000,8.071340e-21,1.479084e-03,-14.748500,-1.474702e+01


In [70]:
shape_df = gdp.read_file(filepath_shp)
shape_df.geometry[0].coords[4]

IndexError: index out of range

In [9]:
test['Resultant_flow'] = test.apply(lambda x: hypot(x['P flux <Flow Flux> (meter pow 3 per sec per meter)'],x['Q flux <Flow Flux> (meter pow 3 per sec per meter)']),axis=1)
test['degrees'] = test.apply(lambda x:degrees(atan2(x['P flux <Flow Flux> (meter pow 3 per sec per meter)'],x['Q flux <Flow Flux> (meter pow 3 per sec per meter)'])),axis=1)

In [21]:
test.columns[2]

'P flux <Flow Flux> (meter pow 3 per sec per meter)'

#### NOTE TO AFTERNOON! if angle == (perpendicular+- 180): positive ,else negative (sort of) 


- No we have to extract at which SIDE the extraction point is from and then done. 

In [11]:
test2=ds.sel(x = lineList[0][0],y=lineList[0][1])

In [19]:
50/4

12.5

In [13]:
test2.geometry.x

147275.75

In [14]:
extracted_point_x = test2.geometry.x
extracted_point_y = test2.geometry.y

In [15]:
extracted_point_y

6395986.750004

In [16]:
atan2(x,y)

-0.38227428775020417

In [17]:
angle_line

-158.7026459548892

In [18]:
import shapely.wkt
import shapely.ops


def reverse_geom(geom):
    def _reverse(x, y, z=None):
        if z:
            return x[::-1], y[::-1], z[::-1]
        return x[::-1], y[::-1]

    return shapely.ops.transform(_reverse, geom)

test3 = reverse_geom(line)

In [19]:
test3.wkt

'LINESTRING (147230.33088362962 6395969.038813498, 147277.16222729348 6395987.295100007)'

In [20]:
line.wkt

'LINESTRING (147277.16222729348 6395987.295100007, 147230.33088362962 6395969.038813498)'

In [37]:
-7.829035669739706/-8.36763

0.935633586779017